# Import library

In [1]:
import cv2 as cv 
import numpy as np
import matplotlib.pyplot as plt
import os 
import time
import mediapipe as mp


2023-07-08 16:39:42.701981: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Make Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic Model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image.flags.writeable = False
    result = model.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    return image, result

In [4]:
def draw_landmarks(image, result):
    mp_drawing.draw_landmarks(image, result.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, result.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def style_draw_landmark(image, result):
    mp_drawing.draw_landmarks(image, result.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    
    mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, result.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

In [6]:
mp_holistic.FACEMESH_TESSELATION

frozenset({(18, 17),
           (82, 38),
           (8, 9),
           (456, 248),
           (167, 2),
           (303, 271),
           (69, 104),
           (253, 450),
           (41, 42),
           (315, 16),
           (73, 74),
           (339, 373),
           (258, 385),
           (105, 63),
           (219, 48),
           (304, 272),
           (236, 217),
           (231, 22),
           (85, 84),
           (365, 367),
           (237, 218),
           (243, 112),
           (436, 427),
           (409, 410),
           (248, 456),
           (330, 266),
           (136, 135),
           (88, 95),
           (394, 430),
           (107, 108),
           (118, 117),
           (439, 438),
           (442, 443),
           (260, 466),
           (115, 48),
           (342, 276),
           (121, 128),
           (200, 421),
           (113, 124),
           (132, 137),
           (23, 230),
           (343, 357),
           (443, 444),
           (62, 78),
           (26,

In [6]:
cap = cv.VideoCapture(0)

# set mediapie model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        #reade frame
        ret, frame = cap.read()
    
        #make detection
        image, result = mediapipe_detection(frame, holistic)
        print(result)
        
        #draw landmark
        style_draw_landmark(image, result)
        
        cv.imshow("Apa Ajalah", image)
        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# Extract Keypoints Values

In [7]:
result.pose_landmarks.landmark[0]

x: 0.5127621
y: 0.40387136
z: -0.62490815
visibility: 0.9999724

In [8]:
pose = []
for res in result.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [11]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in result.pose_landmarks.landmark]).flatten() if result.pose_landmarks else np.zeros(123)
lh = np.array([[res.x, res.y, res.z] for res in result.left_hand_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in result.right_hand_landmarks.landmark]).flatten() if result.right_hand_landmarks else np.zeros(21*3)

In [12]:
def extract_keypoints(result):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in result.pose_landmarks.landmark]).flatten() if result.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in result.face_landmarks.landmark]).flatten() if result.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in result.left_hand_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in result.right_hand_landmarks.landmark]).flatten() if result.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, rh, lh])

In [13]:
extract_keypoints(result).shape

(1662,)

In [14]:
468*3+33*4+21*3+21*3

1662

# Setup Folders For Collections

In [15]:
# path for export data
Path = os.path.join('Dataset')

# actions detect
actions = np.array(['aku', 'terimakasih', 'dimana'])

# thirty video worth of data
no_squence = 30

# videos frame
squence_length = 30

In [16]:
for action in actions:
    for squence in range(no_squence):
        try:
            os.makedirs(os.path.join(Path, action, str(squence)))
        except:
            pass

# Collect Keypoints Values for Tranining and Testing

In [17]:
cap = cv.VideoCapture(0)

# set mediapie model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through action
    for action in actions:
        #Loop through squence aka videos
        for squence in range(no_squence):
            #Loop through video length aka squence length
            for frame_num in range(squence_length):
                
                #reade frame
                ret, frame = cap.read()

                #make detection
                image, result = mediapipe_detection(frame, holistic)
                print(result)

                #draw landmark
                style_draw_landmark(image, result)
                
                #New apply wait logic
                if frame_num == 0:
                    cv.putText(image, 'STARTING COLLECT', (120, 200),
                              cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv.LINE_AA)
                    cv.putText(image, 'Collecting frames for {} video number {}'.format(action, squence), (15, 12),
                              cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv.LINE_AA)
                    cv.imshow("Apa Ajalah", image)
                    cv.waitKey(2000)
                else:
                    cv.putText(image, 'Collecting frames for {} video number {}'.format(action, squence), (15, 12),
                              cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv.LINE_AA)
                
                    cv.imshow("Apa Ajalah", image)
                
                #Exteract Keypoints
                keypoints = extract_keypoints(result)
                npy_path = os.path.join(Path, action, str(squence), str(frame_num))
                np.save(npy_path, keypoints)

                
                if cv.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [18]:
cap.release()
cv.destroyAllWindows()

# Preprocessing Data And Create Labels Features

In [19]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [20]:
label_map = {label:num for num, label in enumerate(actions)}

In [21]:
label_map

{'aku': 0, 'terimakasih': 1, 'dimana': 2}

In [22]:
squences, labels = [], []
for action in actions:
    for squence in range(no_squence):
        window = []
        for frame_num in range(squence_length):
            res = np.load(os.path.join(Path, action, str(squence), "{}.npy".format(frame_num)))
            window.append(res)
        squences.append(window)
        labels.append(label_map[action])

In [23]:
np.array(squences).shape

(90, 30, 1662)

In [24]:
np.array(labels).shape

(90,)

In [25]:
x = np.array(squences)

In [26]:
x.shape

(90, 30, 1662)

In [27]:
y = to_categorical(labels).astype(int)

In [28]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

In [30]:
print(x_train.shape, x_test.shape)

(85, 30, 1662) (5, 30, 1662)


# Build and Train LSTM Nueral Network

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [32]:
log_dir = os.path.join('Logs')
tg_callbacks = TensorBoard(log_dir=log_dir)


In [33]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(36, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [34]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [35]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tg_callbacks])

Epoch 1/2000
3/3 [==============================] - 7s 76ms/step - loss: 3.0194 - categorical_accuracy: 0.2706
Epoch 2/2000
3/3 [==============================] - 0s 75ms/step - loss: 3.5342 - categorical_accuracy: 0.2941
Epoch 3/2000
3/3 [==============================] - 0s 70ms/step - loss: 1.0967 - categorical_accuracy: 0.3059
Epoch 4/2000
3/3 [==============================] - 0s 70ms/step - loss: 1.1596 - categorical_accuracy: 0.3647
Epoch 5/2000
3/3 [==============================] - 0s 73ms/step - loss: 1.1507 - categorical_accuracy: 0.3647
Epoch 6/2000
3/3 [==============================] - 0s 73ms/step - loss: 1.1287 - categorical_accuracy: 0.3294
Epoch 7/2000
3/3 [==============================] - 0s 69ms/step - loss: 1.0806 - categorical_accuracy: 0.3529
Epoch 8/2000
3/3 [==============================] - 0s 73ms/step - loss: 1.1113 - categorical_accuracy: 0.3059
Epoch 9/2000
3/3 [==============================] - 0s 72ms/step - loss: 1.0906 - categorical_accuracy: 0.3882
E

3/3 [==============================] - 0s 88ms/step - loss: 1.0681 - categorical_accuracy: 0.5882
Epoch 74/2000
3/3 [==============================] - 0s 83ms/step - loss: 1.0634 - categorical_accuracy: 0.4941
Epoch 75/2000
3/3 [==============================] - 0s 86ms/step - loss: 1.0565 - categorical_accuracy: 0.5529
Epoch 76/2000
3/3 [==============================] - 0s 76ms/step - loss: 1.0498 - categorical_accuracy: 0.6118
Epoch 77/2000
3/3 [==============================] - 0s 86ms/step - loss: 1.0410 - categorical_accuracy: 0.5529
Epoch 78/2000
3/3 [==============================] - 0s 86ms/step - loss: 1.0293 - categorical_accuracy: 0.5882
Epoch 79/2000
3/3 [==============================] - 0s 89ms/step - loss: 1.0170 - categorical_accuracy: 0.5647
Epoch 80/2000
3/3 [==============================] - 0s 87ms/step - loss: 1.0004 - categorical_accuracy: 0.5765
Epoch 81/2000
3/3 [==============================] - 0s 93ms/step - loss: 0.9766 - categorical_accuracy: 0.6118
Epoch 

3/3 [==============================] - 0s 75ms/step - loss: 1.0995 - categorical_accuracy: 0.3529
Epoch 146/2000
3/3 [==============================] - 0s 70ms/step - loss: 1.0990 - categorical_accuracy: 0.3529
Epoch 147/2000
3/3 [==============================] - 0s 71ms/step - loss: 1.0985 - categorical_accuracy: 0.3529
Epoch 148/2000
3/3 [==============================] - 0s 86ms/step - loss: 1.0985 - categorical_accuracy: 0.3529
Epoch 149/2000
3/3 [==============================] - 0s 97ms/step - loss: 1.0981 - categorical_accuracy: 0.3529
Epoch 150/2000
3/3 [==============================] - 0s 98ms/step - loss: 1.0980 - categorical_accuracy: 0.3529
Epoch 151/2000
3/3 [==============================] - 0s 86ms/step - loss: 1.0979 - categorical_accuracy: 0.3529
Epoch 152/2000
3/3 [==============================] - 0s 92ms/step - loss: 1.0979 - categorical_accuracy: 0.3529
Epoch 153/2000
3/3 [==============================] - 0s 96ms/step - loss: 1.0976 - categorical_accuracy: 0.352

3/3 [==============================] - 0s 69ms/step - loss: 1.0957 - categorical_accuracy: 0.3529
Epoch 218/2000
3/3 [==============================] - 0s 69ms/step - loss: 1.0957 - categorical_accuracy: 0.3529
Epoch 219/2000
3/3 [==============================] - 0s 68ms/step - loss: 1.0957 - categorical_accuracy: 0.3529
Epoch 220/2000
3/3 [==============================] - 0s 76ms/step - loss: 1.0956 - categorical_accuracy: 0.3529
Epoch 221/2000
3/3 [==============================] - 0s 87ms/step - loss: 1.0956 - categorical_accuracy: 0.3529
Epoch 222/2000
3/3 [==============================] - 0s 90ms/step - loss: 1.0956 - categorical_accuracy: 0.3529
Epoch 223/2000
3/3 [==============================] - 0s 87ms/step - loss: 1.0955 - categorical_accuracy: 0.3529
Epoch 224/2000
3/3 [==============================] - 0s 84ms/step - loss: 1.0954 - categorical_accuracy: 0.3529
Epoch 225/2000
3/3 [==============================] - 0s 93ms/step - loss: 1.0958 - categorical_accuracy: 0.352

3/3 [==============================] - 0s 95ms/step - loss: 1.1010 - categorical_accuracy: 0.3059
Epoch 290/2000
3/3 [==============================] - 0s 95ms/step - loss: 1.1005 - categorical_accuracy: 0.4118
Epoch 291/2000
3/3 [==============================] - 0s 92ms/step - loss: 1.0996 - categorical_accuracy: 0.5529
Epoch 292/2000
3/3 [==============================] - 0s 89ms/step - loss: 1.0990 - categorical_accuracy: 0.6235
Epoch 293/2000
3/3 [==============================] - 0s 111ms/step - loss: 1.0985 - categorical_accuracy: 0.6588
Epoch 294/2000
3/3 [==============================] - 0s 130ms/step - loss: 1.0979 - categorical_accuracy: 0.5765
Epoch 295/2000
3/3 [==============================] - 1s 178ms/step - loss: 1.0974 - categorical_accuracy: 0.4706
Epoch 296/2000
3/3 [==============================] - 0s 167ms/step - loss: 1.0963 - categorical_accuracy: 0.4588
Epoch 297/2000
3/3 [==============================] - 0s 125ms/step - loss: 1.0952 - categorical_accuracy: 

...

3/3 [==============================] - 0s 96ms/step - loss: 3.2257e-08 - categorical_accuracy: 1.0000
Epoch 1703/2000
3/3 [==============================] - 0s 93ms/step - loss: 3.2257e-08 - categorical_accuracy: 1.0000
Epoch 1704/2000
3/3 [==============================] - 0s 85ms/step - loss: 3.2257e-08 - categorical_accuracy: 1.0000
Epoch 1705/2000
3/3 [==============================] - 0s 74ms/step - loss: 3.2257e-08 - categorical_accuracy: 1.0000
Epoch 1706/2000
3/3 [==============================] - 0s 74ms/step - loss: 3.2257e-08 - categorical_accuracy: 1.0000
Epoch 1707/2000
3/3 [==============================] - 0s 76ms/step - loss: 3.2257e-08 - categorical_accuracy: 1.0000
Epoch 1708/2000
3/3 [==============================] - 0s 82ms/step - loss: 3.2257e-08 - categorical_accuracy: 1.0000
Epoch 1709/2000
3/3 [==============================] - 0s 74ms/step - loss: 3.2257e-08 - categorical_accuracy: 1.0000
Epoch 1710/2000
3/3 [==============================] - 0s 74ms/step - lo

3/3 [==============================] - 0s 85ms/step - loss: 2.8049e-08 - categorical_accuracy: 1.0000
Epoch 1772/2000
3/3 [==============================] - 0s 69ms/step - loss: 2.6647e-08 - categorical_accuracy: 1.0000
Epoch 1773/2000
3/3 [==============================] - 0s 69ms/step - loss: 2.6647e-08 - categorical_accuracy: 1.0000
Epoch 1774/2000
3/3 [==============================] - 0s 89ms/step - loss: 2.6647e-08 - categorical_accuracy: 1.0000
Epoch 1775/2000
3/3 [==============================] - 0s 94ms/step - loss: 2.6647e-08 - categorical_accuracy: 1.0000
Epoch 1776/2000
3/3 [==============================] - 0s 94ms/step - loss: 2.6647e-08 - categorical_accuracy: 1.0000
Epoch 1777/2000
3/3 [==============================] - 0s 81ms/step - loss: 2.6647e-08 - categorical_accuracy: 1.0000
Epoch 1778/2000
3/3 [==============================] - 0s 92ms/step - loss: 2.6647e-08 - categorical_accuracy: 1.0000
Epoch 1779/2000
3/3 [==============================] - 0s 86ms/step - lo

3/3 [==============================] - 0s 95ms/step - loss: 2.3842e-08 - categorical_accuracy: 1.0000
Epoch 1841/2000
3/3 [==============================] - 0s 93ms/step - loss: 2.2439e-08 - categorical_accuracy: 1.0000
Epoch 1842/2000
3/3 [==============================] - 0s 90ms/step - loss: 2.2439e-08 - categorical_accuracy: 1.0000
Epoch 1843/2000
3/3 [==============================] - 0s 92ms/step - loss: 2.2439e-08 - categorical_accuracy: 1.0000
Epoch 1844/2000
3/3 [==============================] - 0s 91ms/step - loss: 2.2439e-08 - categorical_accuracy: 1.0000
Epoch 1845/2000
3/3 [==============================] - 0s 87ms/step - loss: 2.2439e-08 - categorical_accuracy: 1.0000
Epoch 1846/2000
3/3 [==============================] - 0s 138ms/step - loss: 2.2439e-08 - categorical_accuracy: 1.0000
Epoch 1847/2000
3/3 [==============================] - 0s 176ms/step - loss: 2.3842e-08 - categorical_accuracy: 1.0000
Epoch 1848/2000
3/3 [==============================] - 1s 146ms/step -

3/3 [==============================] - 0s 92ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1910/2000
3/3 [==============================] - 0s 95ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1911/2000
3/3 [==============================] - 0s 95ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1912/2000
3/3 [==============================] - 0s 112ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1913/2000
3/3 [==============================] - 1s 187ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1914/2000
3/3 [==============================] - 0s 144ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1915/2000
3/3 [==============================] - 0s 127ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1916/2000
3/3 [==============================] - 0s 109ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1917/2000
3/3 [==============================] - 0s 107ms/ste

3/3 [==============================] - 0s 97ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1979/2000
3/3 [==============================] - 0s 129ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1980/2000
3/3 [==============================] - 1s 162ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1981/2000
3/3 [==============================] - 0s 138ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1982/2000
3/3 [==============================] - 0s 124ms/step - loss: 1.8232e-08 - categorical_accuracy: 1.0000
Epoch 1983/2000
3/3 [==============================] - 0s 113ms/step - loss: 1.6830e-08 - categorical_accuracy: 1.0000
Epoch 1984/2000
3/3 [==============================] - 0s 111ms/step - loss: 1.6830e-08 - categorical_accuracy: 1.0000
Epoch 1985/2000
3/3 [==============================] - 0s 105ms/step - loss: 1.6830e-08 - categorical_accuracy: 1.0000
Epoch 1986/2000
3/3 [==============================] - 0s 115ms/s

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 36)                2340      
                                                                 
 dense_2 (Dense)             (None, 3)                 111       
                                                                 
Total params: 596,947
Trainable params: 596,947
Non-trai

# Make Prediction

In [37]:
model.predict(x_test)

1/1 [==============================] - 0s 445ms/step


array([[1.0000000e+00, 7.9543891e-13, 2.4267500e-26],
       [4.3180674e-35, 4.1554687e-11, 1.0000000e+00],
       [1.0000000e+00, 2.6313788e-12, 6.3657486e-25],
       [1.0000000e+00, 9.7860487e-12, 1.8656736e-24],
       [1.0000000e+00, 4.3548832e-12, 2.2075540e-23]], dtype=float32)

In [38]:
actions[np.argmax(res[0])]

'aku'

In [39]:
actions[np.argmax(y_test[0])]

'aku'

In [40]:
model.save('detection1.h5')

In [41]:
del model

In [43]:
model.load_weights('detection1.h5')

NameError: name 'model' is not defined

# Evaluate Model

In [24]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [25]:
yhat = model.predict(x_train)

3/3 [==============================] - 1s 33ms/step


In [26]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [27]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[56,  0],
        [21,  8]],

       [[57,  0],
        [ 1, 27]],

       [[35, 22],
        [ 0, 28]]])

In [28]:
accuracy_score(ytrue, yhat)

0.7411764705882353

# Testing

In [29]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv.putText(output_frame, actions[num], (0, 85+num*40), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv.LINE_AA)
        
    return output_frame

In [170]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

IndexError: list index out of range

<Figure size 1296x1296 with 0 Axes>

In [171]:
sequence.reverse()

NameError: name 'sequence' is not defined

In [31]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        style_draw_landmark(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv.putText(image, ' '.join(sentence), (3,30), 
                       cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
        
        # Show to screen
        cv.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 39ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 43ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 82ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 38ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 45ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 36ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 47ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 35ms/step
ilov

1/1 [==============================] - 0s 31ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 41ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 37ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 38ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 108ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 63ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 42ms/step
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
ilo

In [ ]:
model.predict(x_test[0])